In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from pandas.api.types import is_numeric_dtype
from scipy.io.arff import loadarff

raw_data = loadarff('house.arff')
df = pd.DataFrame(raw_data[0])

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#print(df['Misc_Feature'].unique())
#Ridondanze: Pool_QC, Misc_Feature(?),
#TODO CONTROLLARE in base anche a feature selection
#df.drop(columns=['Pool_QC'], inplace=True)
#df.drop(columns=['Misc_Val'], inplace=True)

df.drop_duplicates(subset=['Longitude','Latitude'], keep='last',inplace=True)
df.drop(["Latitude","Longitude", "Pool_QC", "Misc_Feature"],axis = 1 , inplace=True)

columns_to_drop = []
for column in df:
    val = df[column].value_counts().idxmax()
    if df[column].eq(val).mean() > 0.9:
        columns_to_drop.append(column)
df.drop(columns_to_drop, axis=1, inplace=True)

# drop label columns
X = df.drop(columns=['Sale_Price'])

# isolate y
y = df.loc[:, 'Sale_Price']

#splitter=StratifiedShuffleSplit(n_splits=1,random_state=12) #we can make a number of combinations of split
skf = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)

#But we are interested in only one.

for train,test in skf.split(X,y):     #this will splits the index
    X_train_SS = X.iloc[train]
    y_train_SS = y.iloc[train]
    X_test_SS = X.iloc[test]
    y_test_SS = y.iloc[test]

#X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, random_state=10)

#encoded_X_train_80 = pd.get_dummies(X_train_SS)
#encoded_X_test = pd.get_dummies(X_test_SS)
#display(encoded_X_train_80)
#display(encoded_X_test)

#df = df.select_dtypes([np.object])
#df = df.stack().str.decode('utf-8').unstack()
#print(df.dtypes)

is_numerical = np.array([is_numeric_dtype(X_train_SS[col]) for col in X_train_SS])
#numerical_idx = np.flatnonzero(is_numerical)
categorical_idx = np.flatnonzero(is_numerical==False)
#print(numerical_idx, len(numerical_idx))
print(len(categorical_idx))
print(categorical_idx)

oh = OneHotEncoder(sparse=False, handle_unknown='ignore')
oh.fit(X_train_SS.iloc[:, categorical_idx])

encoded = oh.transform(X_train_SS.iloc[:, categorical_idx])

for i,col in enumerate(oh.get_feature_names()):
    X_train_SS[col] = encoded[:, i]

encoded = oh.transform(X_test_SS.iloc[:, categorical_idx])
for i,col in enumerate(oh.get_feature_names()):
    X_test_SS[col] = encoded[:, i]

X_train_SS.drop(columns=X_train_SS.columns[categorical_idx], inplace=True)
X_test_SS.drop(columns=X_test_SS.columns[categorical_idx], inplace=True)

display(X_train_SS)

#----------------------CORRELATIONS------------------
"""
cor_matrix = X_train_SS.corr().abs()
print(cor_matrix)

upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
print(upper_tri)

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print("TO DROP", to_drop)

X_train = X_train_SS.drop(columns=X_train_SS.columns[to_drop])
print(X_train.head())
"""
#-----------------------------------------------------

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_SS,y_train_SS)
best_features = np.argsort(rf.feature_importances_)[::-1]

sum = 0

for bf in best_features:
    sum += rf.feature_importances_[bf]

print(sum)

from sklearn import datasets, linear_model

lasso = linear_model.Lasso()
print(cross_val_score(lasso, X_train_SS, y_train_SS))


# instantiate the DecisionTreeClassifier model with criterion gini index
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=100, random_state=0)

X_train = X_train_SS.drop(columns=X_train_SS.columns[best_features[10:]])
X_test = X_test_SS.drop(columns=X_test_SS.columns[best_features[10:]])
display(X_train)


# fit the model
clf_gini.fit(X_train, y_train_SS)

y_pred_gini = clf_gini.predict(X_test)

from sklearn.metrics import accuracy_score

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test_SS, y_pred_gini)))
print(clf_gini)


C:\Users\matti\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\matti\AppData\Local\Temp/ipykernel_4464/3493426089.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_SS[col] = encoded[:, i]
C:\Users\matti\AppData\Local\Temp/ipykernel_4464/3493426089.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_SS[col] = encoded

32
[ 0  1  4  5  6  7  8  9 10 11 12 15 16 17 18 20 21 22 23 24 25 26 28 32
 40 43 44 45 48 52 55 56]


,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Bsmt_Full_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Cars,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Mo_Sold,Year_Sold,x0_b'Duplex_All_Styles_and_Ages',x0_b'One_Story_1945_and_Older',x0_b'One_Story_1946_and_Newer_All_Styles',x0_b'One_Story_PUD_1946_and_Newer',x0_b'One_Story_with_Finished_Attic_All_Ages',x0_b'One_and_Half_Story_Finished_All_Ages',x0_b'One_and_Half_Story_Unfinished_All_Ages',x0_b'PUD_Multilevel_Split_Level_Foyer',x0_b'Split_Foyer',x0_b'Split_or_Multilevel',x0_b'Two_Family_conversion_All_Styles_and_Ages',x0_b'Two_Story_1945_and_Older',x0_b'Two_Story_1946_and_Newer',x0_b'Two_Story_PUD_1946_and_Newer',x0_b'Two_and_Half_Story_All_Ages',x1_b'A_agr',x1_b'C_all',x1_b'Floating_Village_Residential',x1_b'I_all',x1_b'Residential_High_Density',x1_b'Residential_Low_Density',x1_b'Residential_Medium_Density',x2_b'Irregular',x2_b'Moderately_Irregular',x2_b'Regular',x2_b'Slightly_Irregular',x3_b'Bnk',x3_b'HLS',x3_b'Low',x3_b'Lvl',x4_b'Corner',x4_b'CulDSac',x4_b'FR2',x4_b'FR3',x4_b'Inside',x5_b'Bloomington_Heights',x5_b'Briardale',x5_b'Brookside',x5_b'Clear_Creek',x5_b'College_Creek',x5_b'Crawford',x5_b'Edwards',x5_b'Gilbert',x5_b'Green_Hills',x5_b'Greens',x5_b'Iowa_DOT_and_Rail_Road',x5_b'Landmark',x5_b'Meadow_Village',x5_b'Mitchell',x5_b'North_Ames',x5_b'Northpark_Villa',x5_b'Northridge',x5_b'Northridge_Heights',x5_b'Northwest_Ames',x5_b'Old_Town',x5_b'Sawyer',x5_b'Sawyer_West',x5_b'Somerset',x5_b'South_and_West_of_Iowa_State_University',x5_b'Stone_Brook',x5_b'Timberland',x5_b'Veenker',x6_b'Artery',x6_b'Feedr',x6_b'Norm',x6_b'PosA',x6_b'PosN',x6_b'RRAe',x6_b'RRAn',x6_b'RRNe',x6_b'RRNn',x7_b'Duplex',x7_b'OneFam',x7_b'Twnhs',x7_b'TwnhsE',x7_b'TwoFmCon',x8_b'One_Story',x8_b'One_and_Half_Fin',x8_b'One_and_Half_Unf',x8_b'SFoyer',x8_b'SLvl',x8_b'Two_Story',x8_b'Two_and_Half_Fin',x8_b'Two_and_Half_Unf',x9_b'Above_Average',x9_b'Average',x9_b'Below_Average',x9_b'Excellent',x9_b'Fair',x9_b'Good',x9_b'Poor',x9_b'Very_Excellent',x9_b'Very_Good',x9_b'Very_Poor',x10_b'Above_Average',x10_b'Average',x10_b'Below_Average',x10_b'Excellent',x10_b'Fair',x10_b'Good',x10_b'Poor',x10_b'Very_Good',x10_b'Very_Poor',x11_b'Flat',x11_b'Gable',x11_b'Gambrel',x11_b'Hip',x11_b'Mansard',x11_b'Shed',x12_b'AsbShng',x12_b'AsphShn',x12_b'BrkComm',x12_b'BrkFace',x12_b'CemntBd',x12_b'HdBoard',x12_b'ImStucc',x12_b'MetalSd',x12_b'Plywood',x12_b'PreCast',x12_b'Stone',x12_b'Stucco',x12_b'VinylSd',x12_b'Wd Sdng',x12_b'WdShing',x13_b'AsbShng',x13_b'AsphShn',x13_b'Brk Cmn',x13_b'BrkFace',x13_b'CmentBd',x13_b'HdBoard',x13_b'ImStucc',x13_b'MetalSd',x13_b'Other',x13_b'Plywood',x13_b'PreCast',x13_b'Stone',x13_b'Stucco',x13_b'VinylSd',x13_b'Wd Sdng',x13_b'Wd Shng',x14_b'BrkCmn',x14_b'BrkFace',x14_b'None',x14_b'Stone',x15_b'Excellent',x15_b'Fair',x15_b'Good',x15_b'Typical',x16_b'Excellent',x16_b'Fair',x16_b'Good',x16_b'Poor',x16_b'Typical',x17_b'BrkTil',x17_b'CBlock',x17_b'PConc',x17_b'Slab',x17_b'Stone',x17_b'Wood',x18_b'Excellent',x18_b'Fair',x18_b'Good',x18_b'No_Basement',x18_b'Poor',x18_b'Typical',x19_b'Excellent',x19_b'Fair',x19_b'Good',x19_b'No_Basement',x19_b'Poor',x19_b'Typical',x20_b'Av',x20_b'Gd',x20_b'Mn',x20_b'No',x20_b'No_Basement',x21_b'ALQ',x21_b'BLQ',x21_b'GLQ',x21_b'LwQ',x21_b'No_Basement',x21_b'Rec',x21_b'Unf',x22_b'ALQ',x22_b'BLQ',x22_b'GLQ',x22_b'LwQ',x22_b'No_Basement',x22_b'Rec',x22_b'Unf',x23_b'Excellent',x23_b'Fair',x23_b'Good',x23_b'Poor',x23_b'Typical',x24_b'Excellent',x24_b'Fair',x24_b'Good',x24_b'Typical',x25_b'Excellent',x25_b'Fair',x25_b'Good',x25_b'No_Fireplace',x25_b'Poor',x25_b'Typical',x26_b'Attchd',x26_b'Basment',x26_b'BuiltIn',x26_b'CarPort',x26_b'Detchd',x26_b'More_Than_Two_Types',x26_b'No_Garage',x27_b'Fin',x27_b'No_Garage',x27_b'RFn',x27_b'Unf',x28_b'Excellent',x28_b'Fair',x28_b'Good',x28_b'No_Garage',x28_b'Poor',x28_b'Typical',x29_b'Good_Privacy',x29_

1.0000000000000004


C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 305822820962.68036, tolerance: 669482758.2186916
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 356744291843.44147, tolerance: 717601550.9378058
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 278160111282.1232, tolerance: 703985655.6190223
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

[0.86368164 0.89226449 0.77193197 0.77624645 0.89550986]


C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 349730684420.3919, tolerance: 707818921.5207319
  model = cd_fast.enet_coordinate_descent(


,Lot_Area,Year_Built,Year_Remod_Add,Total_Bsmt_SF,First_Flr_SF,Gr_Liv_Area,Garage_Cars,Garage_Area,x15_b'Good',x24_b'Excellent'
0,31770.0,1960.0,1960.0,1080.0,1656.0,1656.0,2.0,528.0,0.0,0.0
5,9978.0,1998.0,1998.0,926.0,926.0,1604.0,2.0,470.0,0.0,0.0
6,4920.0,2001.0,2001.0,1338.0,1338.0,1338.0,2.0,582.0,1.0,0.0
7,5005.0,1992.0,1992.0,1280.0,1280.0,1280.0,2.0,506.0,1.0,0.0
13,10176.0,1990.0,1990.0,1300.0,1341.0,1341.0,2.0,506.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2920,1894.0,1970.0,1970.0,546.0,546.0,1092.0,1.0,286.0,0.0,0.0
2922,9297.0,1976.0,1976.0,1728.0,1728.0,1728.0,2.0,560.0,0.0,0.0
2925,7937.0,1984.0,1984.0,1003.0,1003.0,1003.0,2.0,588.0,0.0,0.0
2926,8885.0,1983.0,1983.0,864.0,902.0,902.0,2.0,484.0,0.0,0.0


Model accuracy score with criterion gini index: 0.0185
DecisionTreeClassifier(max_depth=100, random_state=0)


In [41]:





#CON HEADER NONE
X = df.loc[:,1:]
# get class label
y = df.loc[:,0]
print("----------------------------")
#SENZA HEADER A NONE PER USARE NOME COLONNE

df = pd.read_csv('house.csv',sep=';')
X = df.loc[:,'MS_SubClass':]
print('-----------------')
y = df.loc[:,'id']

print(type(df['MS_SubClass'][0]))
#TRY TO COMPUTER LABEL ENCODER
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le.fit(df['MS_SubClass'])
df['MS_SubClass'] = le.transform(df['MS_SubClass'])
import numpy as np
for column in df:
    if df[column].dtype == 'object':
        le.fit(df[column])
        df[column] = le.transform(df[column])

print(df.head)
X = df.loc[:,'MS_SubClass':]
print('-----------------')
y = df.loc[:,'id']
feature_names = df.columns[1:]
print(feature_names)


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X,y)
print(rf.feature_importances_)

best_features = np.argsort(rf.feature_importances_)[::-1]

rmse = []
from sklearn import datasets, linear_model

lasso = linear_model.Lasso()
print(cross_val_score(lasso, X, y))

----------------------------
count           2931
unique            29
top       North_Ames
freq             443
Name: 12, dtype: object
count        2931
unique         18
top       VinylSd
freq         1015
Name: 24, dtype: object
2          MS_Zoning
5             Street
6              Alley
7          Lot_Shape
8       Land_Contour
9          Utilities
10        Lot_Config
11        Land_Slope
13       Condition_1
14       Condition_2
15         Bldg_Type
16       House_Style
18      Overall_Cond
21        Roof_Style
22         Roof_Matl
25      Mas_Vnr_Type
27        Exter_Qual
28        Exter_Cond
29        Foundation
30         Bsmt_Qual
31         Bsmt_Cond
32     Bsmt_Exposure
33    BsmtFin_Type_1
34      BsmtFin_SF_1
35    BsmtFin_Type_2
39           Heating
40        Heating_QC
41       Central_Air
42        Electrical
47    Bsmt_Full_Bath
48    Bsmt_Half_Bath
49         Full_Bath
50         Half_Bath
51     Bedroom_AbvGr
52     Kitchen_AbvGr
53      Kitchen_Qual
55        F

In [5]:



import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh import feature_selection

df = pd.read_csv('house.csv',sep=';',header=None)
print(df)




#Misc_Val -> valore della feature
#Misc_Feature -> togliamo ??

#Condition 2 -> 2900 Norm su 2931  -> rimuovere

#Roof_Matl -> rimovere troppo uguali
#Piscina con campo a None
#Street -> 13 diversi da Pavel levare

#Utilities -> levare troppi pochi campi diversi

#anno ristrutturazione come unico Anno

"""""
#decidere se tenere una o eliminare tutte le righe con longitudine e latitudine uguale
df = df.drop_duplicates(subset=['Longitude','Latitude'], keep='last',inplace=False)
df.drop(["Latitude","Longitude"],axis = 1 , inplace=True)


columns_to_drop = []
for column in df:
    val = df[column].value_counts().idxmax()
    if df[column].eq(val).mean() > 0.9:
        columns_to_drop.append(column)
df.drop(columns_to_drop, axis=1, inplace=True)
#print(columns_to_drop)
#print(df.info())


"""""
"""""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(np.unique(df.drop('Lot_Frontage', axis=1)))
X = np.array([le.transform(samp) for samp in df.drop('Lot_Frontage', axis=1).values])
print("-------------------")
print(df['LotFrontage'])
print("-------------------")




# get features by removing id and class
# remove id
X = df.loc[:,]
print("X shape", X.shape)

# get class label
"""""
"""""
y = df.loc[:,]
print("y shape", y.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

le = LabelEncoder()

le.fit(np.unique(df.drop('Lot_Frontage', axis=1)))
X = np.array([le.transform(samp) for samp in df.drop('Lot_Frontage', axis=1).values])
print("-------------------")
print(df['LotFrontage'])
print("-------------------")




X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train)

"""
from sklearn.model_selection import train_test_split
print(df)
X = df.loc[:,1:]
print("X shape", X.shape)

# get class label
y = df.loc[:,0]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X,y)
rf.feature_importances_


fig, ax = plt.subplots(figsize=(9,4))
ax.bar(range(0,X.shape[1]), rf.feature_importances_)
ax.set_title("Feature Importances")
ax.set_xticks(range(X.shape[1]))
ax.set_xticklabels(df.feature_names)
ax.grid();

"""""
for k in range(1,11):

    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit( scaler.transform(X_train), y_train )
    y_pred = kNN.predict( scaler.transform(X_test) )

    # compute Accuracy
    acc = accuracy_score(y_true=y_test, y_pred=y_pred)
    print ("k: {:2d} | Accuracy {:.3f}".format(k,acc) )

"""

ModuleNotFoundError: No module named 'tsfresh'